<a href="https://colab.research.google.com/github/Taeye-Kwack/Predict_illumination/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from pysolar.solar import *
import datetime

In [ ]:
#illuminance
dirpath = '/home/yony/script/ML/FinalProject/rawdata/'
filelist = np.sort(glob.glob(dirpath + 'illum/alltxt/*.txt'))

In [ ]:
alldf = pd.DataFrame()
for ff in range(len(filelist)):
    print(filelist[ff])
    data = np.genfromtxt(filelist[ff],skip_header=1, usecols=(1,2,3),names = ('light','unit','time'),dtype = 'S8,S8,S18')
    df = pd.DataFrame(data)
    df = df.stack().str.decode('utf-8').unstack()

    #light value
    for i in range(len(df)):
        if 'Lux' in df['unit'][i]: #only when the unit is in Lux
            if 'K' in df['light'][i]:
                value = float(df['light'][i].split('K')[0])*1000
                df['light'][i] = value
                #print(value)
            elif 'OL' in df['light'][i]:
                df['light'][i] = np.nan
            else: 
                value = float(df['light'][i])
                df['light'][i] = value
                #print(value)
        else:
            df['light'][i] = np.nan

    df['light'] = df['light'].astype(float)

    #time
    dt = pd.to_datetime(df['time'].values)
    df['time'] = dt
    df = df.set_index(dt)

    #5min average
    df_5min = df['light'].resample('5T').mean()
    df_5min = pd.DataFrame(df_5min)

    #identification of station
    df_5min['CODE'] = filelist[ff][-11:-4] 

    #Solar Zenith Angle
    lon, lat = 37.52475464417339, 126.9828873413339
    sza = np.zeros((len(df_5min)))
    for tt in range(len(df_5min)):
        time = df_5min.index[tt]
        y, m, d, h, mm = time.year, time.month, time.day, time.hour, time.minute
        kst = datetime.timezone(datetime.timedelta(hours=9))
        obstime = datetime.datetime(y,m,d,h,mm,tzinfo =kst)
        angle = 90 - get_altitude_fast(lat, lon, obstime)
        sza[tt] = angle
    df_5min['sza'] = sza
    alldf = pd.concat([alldf,df_5min])
    alldf['time'] = alldf.index

hour = []
for hh in range(len(alldf)):
    hour.append(alldf['time'][hh].strftime('%Y-%m-%d %H'))
alldf['hr'] = np.array(hour)

In [ ]:
alldf['time'][0].strftime('%Y-%m-%d %H')

'2021-06-02 15'

In [ ]:
#temperatur & humidity
meteo = pd.read_csv(dirpath+'aws/OBS_AWS_TIM_RADAR_2020JUNE.csv',engine='python',encoding='CP949' )
meteo.columns = ['NO','sta','time','temp','wd','ws','prcp','pres','seapres','hum']
meteo = meteo[['time','temp','prcp','hum']]

dt = pd.to_datetime(meteo['time'].values)
meteo['time'] = dt
meteo = meteo.set_index(dt)

hour = []
for hh in range(len(meteo)):
    hour.append(meteo['time'][hh].strftime('%Y-%m-%d %H'))
meteo['hr'] = np.array(hour)

#merge
alldf_mt = pd.merge(alldf, meteo, left_on='hr',right_on='hr')

In [ ]:
final = alldf_mt[['time_x','CODE','light','temp','prcp','hum','sza']]

In [ ]:
final.to_csv('./light_meteo_wo_edit.csv', header=True, index=False)

In [ ]:
final

,time_x,CODE,light,temp,prcp,hum,sza
0,2021-06-07 12:15:00,9.32_82,943.000000,16.7,0.0,66.9,70.018247
1,2021-06-07 12:20:00,9.32_82,950.333333,16.7,0.0,66.9,70.753575
2,2021-06-07 12:25:00,9.32_82,951.000000,16.7,0.0,66.9,71.486218
3,2021-06-07 12:30:00,9.32_82,951.000000,16.7,0.0,66.9,72.215985
4,2021-06-07 12:35:00,9.32_82,951.000000,16.7,0.0,66.9,72.942685
...,...,...,...,...,...,...,...
440,2021-06-14 01:20:00,.28_220,200.000000,21.2,0.0,53.4,64.204513
441,2021-06-14 01:25:00,.28_220,200.000000,21.2,0.0,53.4,63.454814
442,2021-06-14 01:30:00,.28_220,200.000000,21.2,0.0,53.4,62.704234
443,2021-06-14 01:35:00,.28_220,200.000000,21.2,0.0,53.4,61.952985
